### A simple A <-> B reaction between 2 species with initial uniform concentrations across 3 bins,
with 1st-order kinetics in both directions, taken to equilibrium

Diffusion NOT taken into account

See also the experiment _"reactions_single_compartment/react_1"_ 

LAST REVISED: Jan. 15, 2023

In [1]:
# Extend the sys.path variable, to contain the project's root directory
import set_path
set_path.add_ancestor_dir_to_syspath(3)  # The number of levels to go up 
                                         # to reach the project's home, from the folder containing this notebook

Added 'D:\Docs\- MY CODE\BioSimulations\life123-Win7' to sys.path


In [2]:
from experiments.get_notebook_info import get_notebook_basename

from modules.reactions.reaction_data import ReactionData as chem
from modules.reactions.reaction_dynamics import ReactionDynamics
from life_1D.bio_sim_1d import BioSim1D

import plotly.express as px
from modules.html_log.html_log import HtmlLog as log
from modules.visualization.graphic_log import GraphicLog

In [3]:
# Initialize the HTML logging (for the graphics)
log_file = get_notebook_basename() + ".log.htm"    # Use the notebook base filename for the log file

# Set up the use of some specified graphic (Vue) components
GraphicLog.config(filename=log_file,
                  components=["vue_cytoscape_1"],
                  extra_js="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.21.2/cytoscape.umd.js")

-> Output will be LOGGED into the file 'reaction_1.log.htm'


# Initialize the System

In [4]:
# Initialize the system
chem_data = chem(names=["A", "B"])              # Diffusion NOT taken into account
bio = BioSim1D(n_bins=3, chem_data=chem_data)   # We'll specify the reactions later

bio.set_uniform_concentration(species_name="A", conc=10.)
bio.set_uniform_concentration(species_name="B", conc=50.)

bio.describe_state()

SYSTEM STATE at Time t = 0:
3 bins and 2 species:
  Species 0 (A). Diff rate: None. Conc: [10. 10. 10.]
  Species 1 (B). Diff rate: None. Conc: [50. 50. 50.]


In [5]:
# Save the state of the concentrations of all species at bin 0
bio.add_snapshot(bio.bin_snapshot(bin_address = 0), caption="Initial state")

In [6]:
bio.get_history()

,SYSTEM TIME,A,B,caption
0,0,10.0,50.0,Initial state


In [7]:
# Specify the reaction

# Reaction A <-> B , with 1st-order kinetics in both directions
chem_data.add_reaction(reactants=["A"], products=["B"], forward_rate=3., reverse_rate=2.)

print("Number of reactions: ", chem_data.number_of_reactions())

Number of reactions:  1


In [8]:
chem_data.describe_reactions()

Number of reactions: 1 (at temp. 25 C)
0: A <-> B  (kF = 3 / kR = 2 / Delta_G = -1,005.13 / K = 1.5) | 1st order in all reactants & products


In [9]:
# Send a plot of the network of reactions to the HTML log file
graph_data = chem_data.prepare_graph_network()
GraphicLog.export_plot(graph_data, "vue_cytoscape_1")

[GRAPHIC ELEMENT SENT TO LOG FILE `reaction_1.log.htm`]


### <a name="sec_1"></a>First step

In [10]:
# First step of reaction
bio.react(time_step=0.1, n_steps=1)
bio.describe_state()

SYSTEM STATE at Time t = 0.1:
3 bins and 2 species:
  Species 0 (A). Diff rate: None. Conc: [17. 17. 17.]
  Species 1 (B). Diff rate: None. Conc: [43. 43. 43.]


NOTE: the concentration of species A is increasing, while that of species B is decreasing.
All bins have identical concentrations; so, there's no diffusion (and we're not attempting to compute it):  
[[17. 17. 17.]  
 [43. 43. 43.]]

In [11]:
# Save the state of the concentrations of all species at bin 0
bio.add_snapshot(bio.bin_snapshot(bin_address = 0))
bio.get_history()

,SYSTEM TIME,A,B,caption
0,0.0,10.0,50.0,Initial state
1,0.1,17.0,43.0,


### <a name="sec_more_steps"></a>Numerous more steps

In [12]:
# Numerous more steps
bio.react(time_step=0.1, n_steps=10, snapshots={"sample_bin": 0})

bio.describe_state()

SYSTEM STATE at Time t = 1.1:
3 bins and 2 species:
  Species 0 (A). Diff rate: None. Conc: [23.99316406 23.99316406 23.99316406]
  Species 1 (B). Diff rate: None. Conc: [36.00683594 36.00683594 36.00683594]


### <a name="sec_equilibrium"></a>Equilibrium

NOTE: Consistent with the 3/2 ratio of forward/reverse rates (and the 1st order reactions),
 the systems settles in the following equilibrium:

[A] = 23.99316406
 
[B] = 36.00683594


In [13]:
bio.bin_snapshot(bin_address = 0)

{'A': 23.9931640625, 'B': 36.0068359375}

In [17]:
# Verify that the reaction has reached equilibrium
bio.reaction_dynamics.is_in_equilibrium(conc=bio.bin_snapshot(bin_address = 0))

A <-> B
Final concentrations:  [B] = 36.01 ; [A] = 23.99
1. Ratio of reactant/product concentrations, adjusted for reaction orders: 1.50071
    Formula used:  [B] / [A]
2. Ratio of forward/reverse reaction rates: 1.5
Discrepancy between the two values: 0.04749%
Reaction IS in equilibrium (within tolerance)



{0: True}

In [ ]:
# Save the state of the concentrations of all species at bin 0
#bio.save_snapshot(bio.bin_snapshot(bin_address = 0))
bio.get_history()

## Plots of changes of concentration with time

In [ ]:
fig = px.line(data_frame=bio.get_history(), x="SYSTEM TIME", y=["A", "B"], 
              title="Changes in concentrations with time",
              color_discrete_sequence = ['navy', 'darkorange'],
              labels={"value":"concentration", "variable":"Chemical"})
fig.show()

In [ ]:
# Same plot, but with a smoothed line
fig = px.line(data_frame=bio.get_history(), x="SYSTEM TIME", y=["A", "B"], 
              title="Changes in concentrations with time (smoothed)",
              color_discrete_sequence = ['navy', 'darkorange'],
              labels={"value":"concentration", "variable":"Chemical"},
              line_shape="spline")
fig.show()

## For more in-depth analysis of this reaction, see the experiment _"reactions_single_compartment/react_1"_ 